In [1]:
from DataProcessing import DataProcessing
from Car import Car
from Cba import Cba
from Validation import Validation

## 原版 CBA


In [2]:
data_procesing = DataProcessing("car.data")
data_procesing.read_data()
data_procesing.process()
data = data_procesing.encoded_data
train_data, test_data = data_procesing.split_train_test_data()
MIN_SUPPORT = 0.1
MIN_CONFIDENCE = 0.8
car = Car(train_data, MIN_SUPPORT, MIN_CONFIDENCE)
car.generate_frequent()
car.sort_rule()
cba = Cba(train_data, car.rule)
cba.cover()
rules = cba.final_rules_
default = cba.default
test = Validation(test_data, rules, default)

              precision    recall  f1-score   support

           0       0.37      0.84      0.51        63
           1       0.00      0.00      0.00        46
           2       0.95      0.97      0.96       198
           3       0.00      0.00      0.00        39

    accuracy                           0.71       346
   macro avg       0.33      0.45      0.37       346
weighted avg       0.61      0.71      0.64       346



## CBA 加入 LIFT


In [17]:
train_data, test_data = data_procesing.split_train_test_data()
MIN_SUPPORT = 0.1
MIN_CONFIDENCE = 0.8
MIN_LIFT = 1.4
car = Car(train_data, MIN_SUPPORT, MIN_CONFIDENCE, MIN_LIFT)
car.generate_frequent()
car.sort_rule()
cba = Cba(train_data, car.rule)
cba.cover()
rules = cba.final_rules_
default = cba.default
test = Validation(test_data, rules, default)

              precision    recall  f1-score   support

           0       0.27      1.00      0.42        63
           1       0.00      0.00      0.00        46
           2       1.00      0.57      0.72       198
           3       0.00      0.00      0.00        39

    accuracy                           0.51       346
   macro avg       0.32      0.39      0.29       346
weighted avg       0.62      0.51      0.49       346



In [18]:
data_procesing = DataProcessing("car.data")
data_procesing.read_data()
data_procesing.process()
data_procesing.get_feature_importances()
print(data_procesing.feature_importances)

[0.19075291 0.15874729 0.05813035 0.22113893 0.08141021 0.28982031]
